# Crawling PTA Trunojoyo Fakultas Ekonomi Prodi Manajemen

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [38]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

# Fungsi untuk menentukan maksimal halaman pada daftar page pada website

In [39]:
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        max_page = int(href.split("/")[-1])
        return max_page
    return 1

# Fungsi untuk menampilkan progres bar pada terminal atau console

In [40]:
def print_progress(current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\rPage {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

# Fungsi untuk mengambil detail isi skripsi dan fungsi untuk crawling seluruh data skripsi dari prodi Manajemen

In [ ]:
def crawl_detail(link_keluar, prodi):
    try:
        id_match = re.search(r"/detail/(\d+)", link_keluar)
        pta_id = id_match.group(1) if id_match else None

        response = requests.get(link_keluar, timeout=10)
        soup1 = BeautifulSoup(response.content, "html.parser")
        isi = soup1.select_one('div#content_journal')

        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

        paragraf = isi.select('p[align="justify"]')
        abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
        abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

        return {
            "id": pta_id,
            "penulis": penulis,
            "judul": judul,
            "abstrak_id": abstrak_id,
            "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua,
            "prodi": prodi,
        }
    except Exception:
        return None

def crawl_manajemen():
    start_time = time.time()
    results = []

    prodi_id = 7 # id prodi manajemen
    max_page = get_max_page(prodi_id)

    for j in range(1, max_page + 1):
        url = f"{BASE_URL}/{prodi_id}/{j}"
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        jurnals = soup.select('li[data-cat="#luxury"]')

        isii = soup.select_one('div#begin')
        if not isii:
            continue
        prodi_full = isii.select_one('h2').text.strip()
        prodi = prodi_full.replace("Journal Jurusan ", "")

        links = [jurnal.select_one('a.gray.button')['href'] for jurnal in jurnals]

        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_link = {executor.submit(crawl_detail, link, prodi): link for link in links}
            for future in as_completed(future_to_link):
                data = future.result()
                if data:
                    results.append(data)

        print_progress(j, max_page)

    df = pd.DataFrame(results)
    df.to_csv("pta_manajemen.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Data Prodi Manajemen berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [42]:
crawl_manajemen()

Page 207/207 [████████████████████] 100.00%

✅ Data Prodi Manajemen berhasil dikumpulkan!
📊 Total entri: 1031
⏱️ Waktu eksekusi: 0 jam 7 menit 50 detik


,id,penulis,judul,abstrak_id,pembimbing_pertama,pembimbing_kedua,prodi
0,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
1,090211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,Nurita Andriani,Yustina Chrismardani,Manajemen
2,080211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...","Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
3,080211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
4,080211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
...,...,...,...,...,...,...,...
1026,160211100291,Uswatun Hasanah,Pengaruh Pelatihan Dan Kompensasi Terhadap Pro...,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...","Dr. Raden Mas Mochammad Wispandono S.E ., MS",,Manajemen
1027,160211100041,Novia Prihastiwi Ningtyas,"ANALISA PENGARUH RASIO BOPO, LDR, ROA TERHADAP...",Tujuan penelitian ini adalah untuk mengetahui ...,"Hj. Evaliati Amaniyah, S.E., M.S.M",,Manajemen
1028,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,"DR. MOHAMMAD ARIEF, S.E., M.M.",,Manajemen
1029,160211100064,ACH FATHONI,PERAN SERVICE PERFORMANCE DAN CLIMATE ORGANIZA...,ABSTRAK\nTujuan dari penelitian ini adalah unt...,"YUDHI PRASETYA MADA, S.E., M.M.",,Manajemen
